# Домашнее задание 3

В этом задании напишем простое решение классификации датасета `FashionMNIST`, а затем будем его улучшать с помощью:
- dropout;
- batch normalization;
- LR scheduler;

В конце сохраним модель в файл и убедимся, что этот файл можем в дальнейшем прочитать.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from dataclasses import dataclass
import tqdm
import wandb
import torch.nn.functional as F
from torch.optim.sgd import SGD
import typing as tp


In [2]:
train_dataset = FashionMNIST(
    root="./data", train=True, download=True, transform=ToTensor()
)
test_dataset = FashionMNIST(
    root="./data", train=False, download=True, transform=ToTensor()
)

In [3]:
X_train = train_dataset.data.float()
y_train = train_dataset.targets
X_test = test_dataset.data.float()
y_test = test_dataset.targets

In [16]:
len(y_train.unique())

10

In [4]:



# Для оценки будем использовать метрику accuracy
# Подумайте (опционально), какие еще метрики можно использовать
def calculate_accuracy(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).float().sum()
    accuracy = correct / y_true.shape[0]
    return accuracy.item()

## Задание №1

Попробуйте реализовать простой бейзлайн с несколькими слоями:
- Linear
- ReLU
- Linear

Вставьте свою релизацию `SimpleModel` в проверку.
Вам нужно дописать и сдать как `SimpleModel`, так и `train_loop`.

Используйте кросс-энтропию как функцию потерь.

In [5]:
wandb.login(key = 'cc603ae0565bbbfce5cc5b068a9bddabb8950920')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/theo/.netrc
wandb: Currently logged in as: theotheo46 (theotheo46-trs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import wandb
import torch.nn.functional as F
from torch.optim.sgd import SGD
import tqdm
import typing as tp
import torch.optim.lr_scheduler as lr_scheduler

@dataclass
class TrainConfig:
    lr: float = 1e-2
    total_iterations: int = 200
    step_size: int = 30
    gamma: float = 0.5
    eval_every: int = 10

class ExpModel(nn.Module):
    def __init__(self, num_classes: int):
        super(ExpModel, self).__init__()
        hidden_dim = 512
        dropout_rate = 0.3
        self.net = nn.Sequential(
            nn.Linear(in_features=28 * 28, out_features=hidden_dim),
            nn.Dropout(p=dropout_rate),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=hidden_dim),
            nn.Linear(in_features=hidden_dim, out_features=num_classes),
            # nn.Softmax(dim=1)
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.reshape((-1, 28 * 28))
        return self.net(x)

def train_loop(
    model: ExpModel,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    """Обучите здесь модель, подсчитайте метрики на валидационной выборке.

    Можете так же писать/рисовать accuracy в процессе обучения.
    Например, каждые 10 итераций или даже каждую итерацию.
    """
    wandb.login(key = 'cc603ae0565bbbfce5cc5b068a9bddabb8950920')
    wandb.init(
        project="simple-model-train",
        notes="version 1",
        tags=["sgd", "2-layer"],
        config=config,
    )
    optimizer = SGD(model.parameters(), lr=config.lr)
    scheduler = lr_scheduler.StepLR(optimizer, config.step_size, config.gamma)

    model.train()
    mac_accuracy = 0
    for i in tqdm.trange(config.total_iterations):
        optimizer.zero_grad()
        loss = F.cross_entropy(model(X_train), y_train)
        loss.backward()
        optimizer.step()
        metrics = {"iteration": i, "loss_train": loss.detach().cpu().item()}
        # каждые `eval_every` итераций будем считать метрику на отложенной выборке
        if (i + 1) % config.eval_every == 0:
            with torch.no_grad():
                model.eval()
                accuracy = calculate_accuracy(model(X_val), y_val)
                if accuracy > mac_accuracy:
                    mac_accuracy = accuracy

                model.train()
                metrics.update({"accuracy": accuracy})
                scheduler.step()
                metrics.update({"lr": scheduler.get_last_lr()[0]})


        wandb.log(metrics)
    wandb.finish()
    print(mac_accuracy)

torch.random.manual_seed(987)
model = ExpModel(num_classes=len(y_train.unique()))
config = TrainConfig()
train_loop(model, X_train, y_train, X_test, y_test, config=config)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/theo/.netrc


100%|██████████| 200/200 [01:18<00:00,  2.56it/s]


accuracy,▁▄▅▅▆▆▆▇▇▇▇▇▇███████
iteration,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇███
loss_train,█▇▆▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.8201
iteration,199
loss_train,0.54891
lr,0.01


0.8201000094413757


In [14]:
torch.save(model.state_dict(), "model_weights.pt")

## Задание №2
Какое максимальное значение метрики accuracy удалось получить в процессе обучения?

Округлите до 3 значений после запятой


In [7]:
torch.manual_seed(987)
# дефолтные значения
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №3
Добавьте один `dropout` слой в вашу модель.

_Подумайте, что может поменяться при перестановке ReLU и Dropout слоев местами._

In [8]:
# Возможно, класс нужно отнаследовать от некого класса из pytorch
class DropoutModel:
    hidden_dim = 512
    ...

In [9]:
torch.manual_seed(987)
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №4
Какое максимальное значение accuracy получилось в ходе обучения модели? 

Округлите до 3х знаков после запятой и отправьте в ЛМС.

## Задание №5

Добавьте `BatchNorm` в вашу модель.
Отправьте в ЛМС реализацию.

Стоит ли делать BatchNorm до ReLU или после него?
Это дискуссионный вопрос, чаще всего применяют сначала нелинейность, затем Batch Norm.
Один из аргументов: при таком подходе данные на выходе будут иметь среднее 0 - что и ожидают люди, когда добавляют нормализацию.

_[Дискуссия на Reddit](https://www.reddit.com/r/MachineLearning/comments/67gonq/d_batch_normalization_before_or_after_relu/)_

Для определенности в этом задании будем следовать такому порядку: сначала ReLU, затем Batch Norm.

In [10]:
# Возможно, класс нужно отнаследовать от некого класса из pytorch
class BatchNormModel:
    hidden_dim = 512
    ...

In [11]:
torch.manual_seed(987)
config = TrainConfig()
# Ваш код для обучения и подсчета accuracy

## Задание №6
Какое максимальное значение `accuracy` получилось в ходе обучения модели? 

Округлите до 3х знаков после запятой.

Результат batch normalization мог не особо порадовать.
Но не спешите с выводами насчет этого слоя!

Попробуйте обучить заново все три модели со значением `lr=1e-2` (в 10 раз больше).
Сравните результаты моделей и сделайте вывод.

## Задание №7
Добавьте `LRscheduler` в вашу модель.

Подробнее про `schedulers` можно почитать в [документации](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)

In [13]:
from torch.optim.lr_scheduler import StepLR


def train_loop_with_scheduler(
    model,
    X_train: torch.Tensor,
    y_train: torch.Tensor,
    X_val: torch.Tensor,
    y_val: torch.Tensor,
    config: TrainConfig,
):
    ...
    scheduler = StepLR(..., step_size=5, gamma=0.1)
    ...

In [14]:
torch.manual_seed(987)
config = TrainConfig(lr=1e-3)
# Ваш код для обучения и подсчета accuracy

## Задание №8

Поэксперементируйте с параметрами нейронной сети, попробуйте добиться максимальной метрики `accuracy`.

- попробуйте комбинацию Drouput + Batch Normalization и подумайте, как лучше всего раскрыть силу batch normalization (вспомните эксперименты с lr);
- попробуйте подвигать вероятность в Dropout;
- ну, или подержите обучение подольше, поставив больше шагов :)

В ЛМС нужно сдать код класса `ExpModel`.
Вам необходимо выбить accuracy > 80%, чтобы сдать этот пункт.

In [27]:
torch.manual_seed(987)


# Ваш код модели и ее обучения при seed = 987
class ExpModel: ...


model = ExpModel()
config = TrainConfig(...)
train_loop(model, X_train, y_train, X_test, y_test, config)

Наконец, сохраним лучшую модель, чтобы в будущем ее могли взять и использовать, без обучения.

## Задание №9

Напишите код, который сохранит модель в файл `model.pt`.

In [29]:
# Впоследствии эту модель можно будет загрузить вот так:
model_loaded = ExpModel(num_classes=len(y_test.unique()))
model_loaded.load_state_dict(torch.load("model.pt"))